# Part 1
### Define and explain the graphs being used to model the android applications (from the Hindroid paper). Explain any computationally relevant portions of the process of writing (text processing) code to extract the nodes and edges of the graphs.
The graphs in used to model Android applications are:
* app-to-api: maps apps to all the different APIs that they use
* api-to-api: maps APIs to APIs based on if they coexist in the same code block
* api-to-api: maps APIs to APIs based on if they are in the same package. To save space this is done as a mapping of API<-> packages. In order to get API<->API we can simply do AA<sup>T</sup>
* api-to-api: maps APIs to APIs based on if they are invoked in the same way. To save space this is done as a mapping of API<->invoke method. To get API<->API we can simply do AA<sup>T</sup>

In order to text process, we need to extract call methods, remember in the methods what APIs we have called, and have some way of associating each method call with a respective node in our graph. Then, we can simply get those nodes out to create edges between them.

In [3]:
from data_pipeline import download_and_process_apks

download_and_process_apks(50)

ignoring because size is 71.600000
ignoring because size is 626.600000
ignoring 2048 because exists
ignoring 2048 because exists
4636 downloaded
ignoring because size is 84.900000
five-craft-nights downloaded
modern-sniper downloaded
ignoring guns because exists
dual downloaded
ignoring flappy-crush because exists
batman-the-flash-hero-run downloaded
gun-shoot-war downloaded
ignoring because size is 69.600000
sonic-jump-fever downloaded
ignoring because size is 99.500000
ignoring because size is 89.100000
ignoring because size is 75.700000
prop-hunt-portable downloaded
deathrun-portable downloaded
ignoring because size is 61.700000
ignoring because size is 53.600000
mountain-sniper-shooting-3d-fps downloaded
ignoring ninja-revenge because exists
zombie-frontier downloaded
ignoring birds-vs-zombies-2 because exists
zombie-shooter-3d downloaded
city-battle-war downloaded
ignoring assassin-ape-open-world-game because exists
ignoring because size is 80.100000
101-skateboard-racing-3d downl

### EDA on amount of classes per app

In [5]:
import os

total_classes_regular = 0
total_apps_regular = 0
print('Regular Apps classes per app:')
for directory in next(os.walk('data'))[1]:
    total_apps_regular += 1
    
    classes = sum([len(files) for subdir, directory, files in os.walk('data/' + directory)])
    total_classes_regular += classes
    print(directory + ": " + str(classes))
print('Average classes per app: %i' % (total_classes_regular / total_apps_regular))
        
print('\nMalware:')
for directory in next(os.walk('/datasets/dsc180a-wi20-public/Malware/amd_data_smali'))[1]:
    total_classes_variety = 0
    total_apps_variety = 0
    for variety in next(os.walk('/datasets/dsc180a-wi20-public/Malware/amd_data_smali/' + directory))[1]:
        for sample in next(os.walk('/datasets/dsc180a-wi20-public/Malware/amd_data_smali/%s/%s' % (directory, variety)))[1]:
            total_apps_variety += 1

            classes = sum([len(files) for r, d, files in os.walk('/datasets/dsc180a-wi20-public/Malware/amd_data_smali/%s/%s/%s/' % (directory, variety, sample))])
            total_classes_variety += classes
            print('%s/%s/%s: ' % (directory, variety, sample) + str(classes))
    print('Average classes for %s: %i' % (directory, total_classes_variety / total_apps_variety))

Regular Apps classes per app:
flashlight-tool: 3493
stickman-and-gun: 2748
angry-sea-white-shark-revenge: 7367
tv-remote-control-pro: 1837
trigger-down: 2101
amazing-ninja: 2426
batman-the-flash-hero-run: 7326
zombie-objective: 2671
gun-shoot-war: 6839
nob-s-world-jungle-adventure: 9633
goku-saiyan-warrior: 7273
sniper-shoot-war-3d: 12267
bunny-run: 6288
super-oscar: 8147
city-battle-war: 5056
ninja-revenge: 9110
zombie-diary-2-evolution: 8579
amazing-spider-avenger: 2091
shinobi-war: 4303
zombie-frontier: 8146
modern-sniper: 3916
dragon-warrior-z-doomsday: 7380
4636: 9
killer-bean-unleashed: 5654
top-celebrity-3d-fashion-game: 3987
transformers-legends: 4613
guns: 8796
gunship-battle-bullet-train-3d: 7529
angry-lion-attack-3d: 6395
angry-birds-rio: 7272
pokedex-z: 4270
adam-and-eve-2: 2110
jungle-dinosaurs-hunting-game-3d: 5525
don-t-touch-the-spikes: 3801
reckless-moto-rider: 858
crazy-dentist-fun-games: 15772
benji-bananas: 8676
super-tank-battle-mycityarmy-remaster-90-tv-game: 5378

GingerMaster/variety2/222bc1056fbae2a4f1e9355483502edd: 336
GingerMaster/variety2/dd8ea3ae4854f1a878ad89e23b2d3e23: 336
GingerMaster/variety5/9b12b63050eafb36a0fed4f44aa1b748: 169
GingerMaster/variety5/9313fa19ab5d4b19a5c02eee44068375: 143
GingerMaster/variety5/dd06ba9a1dd456bbbad888bca216234e: 140
GingerMaster/variety5/4c1b964b290849a2332550797abda240: 128
GingerMaster/variety5/b81f568fec464f260d988bdbbb2c063d: 628
GingerMaster/variety5/aa44240b167e1111484ba430360595aa: 172
GingerMaster/variety5/30c637fe93a1b9b4aa9047c6ec981cb7: 287
GingerMaster/variety5/4fd35916a26341a1e55bb3ce253ab48a: 195
GingerMaster/variety5/ab3f5567a9f3a310ac20e0a27dc9610d: 66
GingerMaster/variety5/cdd055be9c54df192fdc7d87997a90c1: 726
GingerMaster/variety5/7849301b637e35e6ada8b9a6bc7f6f0c: 1650
GingerMaster/variety5/0e23aa08241ce68912d72e98d4f76ee4: 170
GingerMaster/variety5/b34c39a7e5830a8194311786dd44236a: 237
GingerMaster/variety5/f232db01feca8ea5563a77ac20d33f18: 110
GingerMaster/variety1/779925c209ddba5d0d

Youmi/variety1/7b708c7070569d0d835b8e5a661d7a7b: 463
Youmi/variety1/a7b54e2363d82cd7fdc483469b88be98: 180
Youmi/variety1/f38922d68e7bc14f7c9c4da284833ad8: 1239
Youmi/variety1/6e94d7fe8f06e66a4410900003d9a40d: 632
Youmi/variety1/2ed88701cf10c506f1b7ea7b5db308e6: 2146
Youmi/variety1/e8b75b40e7eec2791f09abb1eb87deb5: 4763
Youmi/variety1/49d43396403c915b65e806fec2fd09fc: 1336
Youmi/variety1/53fdd4717ffa1e755358ee60d9b9d5ea: 239
Youmi/variety1/f62724cddb173f4f57b367c2b79c32ed: 171
Youmi/variety1/3a8e41b57184800c5fc64cd777f61cd6: 1396
Youmi/variety1/322b86018c55e93b8c8e0f23e17f028c: 5248
Youmi/variety1/57baaddb97967496d2752f173fe77310: 1693
Youmi/variety1/ce915c6af4787c9ead701c152518d1b9: 956
Youmi/variety1/bf20da59f5d420cac43a5436e57d639e: 1263
Youmi/variety1/496360a2824f8efb0d2d72b3c30ba718: 495
Youmi/variety1/9028ff6b0d19f29edfa43ebe62ff950b: 975
Youmi/variety1/3cb2dc7ab43fa615a629be244859c311: 1389
Youmi/variety1/784c50f73c38f07b06242354cb1aa524: 296
Youmi/variety1/0bf6bef249ade12b1bb047

Youmi/variety1/75747556f640986981722e3669c4d5ec: 1680
Youmi/variety1/bf2e8ebb752624d0a291f660c8ffc89d: 2047
Youmi/variety1/9fae4b2a00eca5be973283d265b1171c: 1693
Youmi/variety1/3b2047865ce015936eac328f55b3c7d8: 1693
Youmi/variety1/b9916f0c0c731571e3fa4b16d42a338a: 2248
Youmi/variety1/4d45bed4a39d51098a537cc0c334d674: 1863
Youmi/variety1/0746c882cc27e12f032f947306b84a47: 470
Youmi/variety1/e65e31e8d6a619d4473ec2ef8983f72e: 841
Youmi/variety1/317dc6cf9b87168bd0321110ca34274b: 313
Youmi/variety1/0e2c6208278fd19474d0f12a77ce113a: 116
Youmi/variety1/67338720f1bc91be5ceb4c81aebfc0dd: 1162
Youmi/variety1/d92fc2c5866c2059bf34141527a0e876: 1693
Youmi/variety1/e7218b7e64d18c111b032c4247c50357: 628
Youmi/variety1/bd243ba3bf06bf1cb15f604498885851: 513
Youmi/variety1/1ebc7a41c59e56692bf17916662a3c35: 467
Youmi/variety1/33f981315b17ba21e5acdd16bcdb5125: 810
Youmi/variety1/9640c8d9b29b7c74e3cc300912259a17: 197
Youmi/variety1/871765bc88604a468c3d12fe9059a898: 2562
Youmi/variety1/e3c53b406a851a7612a601

Youmi/variety1/e1b9d155f94f23ef903bde2c939a9bbe: 1689
Youmi/variety1/e2455355cdbcd875c2fc16c8db5e1da1: 172
Youmi/variety1/4aca213e939fbd07e041e7801f01def2: 896
Youmi/variety1/253fdb4c13380541d00297cf9b456243: 1561
Youmi/variety1/f78379dc01f13f6220749dd3d2c0dae3: 534
Youmi/variety1/1ebf010c389052cc95f3e4da33f480ea: 1828
Youmi/variety1/e0f612e83e7dd6bb35a5dc3f000228c5: 5208
Youmi/variety1/4b94dafc8ad8e6dcb3e5f06f4ea205c4: 2424
Youmi/variety1/10615b6961762db0ece1ae88573b982d: 1687
Youmi/variety1/0ddc819c4cd623f98a7b6bb477ace624: 1693
Youmi/variety1/5a384df715064e6e44d17d1d5fe6d0e8: 640
Youmi/variety1/fce490a108c46b964d061da545340d78: 183
Youmi/variety1/9ca4a773483fb5b84717ffe21f933994: 517
Youmi/variety1/3f5588b2780b94fb5ce57ce2b3f1c03e: 1641
Youmi/variety1/a5dc3549fa05c5e33947efc9465ae357: 966
Youmi/variety1/4e0bbb9913f3fb396552ffcd994d26cc: 121
Youmi/variety1/aa183df3c9f269dae4fc716558229474: 437
Youmi/variety1/c3d2e10c55971c67405409942787ed39: 1768
Youmi/variety1/10993e2b875078c018d86e

...

BankBot/variety7/084ba13ddcf17b675ee86b6e85f7cabf: 609
BankBot/variety7/8bc6b49609b0c595aa97b8f31f86c645: 1415
BankBot/variety7/319fc149dd63e02ffc8021afd34bd410: 605
BankBot/variety7/2030a28c3a874010bd73bb92cb95b4a1: 609
BankBot/variety7/33265aa59b03965a16352b229170eedc: 609
BankBot/variety7/2b08fdd40a9d99aeea8af7cd92ac6752: 1415
BankBot/variety7/2d8991403092408e5b7a03e89ed7317d: 603
BankBot/variety7/9add025feea977cf18f9d0236e53f108: 605
BankBot/variety7/bc286727ff5a38e2c9d4c6979e2cb296: 1415
BankBot/variety7/13b23f093555add17df70ba7df560c46: 607
BankBot/variety7/371e1dadaf4eea5fdaece63a080b6b9b: 1415
BankBot/variety7/c2f64d7f971ff479eb7cae06c8357ee8: 1415
BankBot/variety7/5bf8dbd812d248b8d510e35d27617458: 1415
BankBot/variety7/58229dcf4519f930f44c908d1c683071: 609
BankBot/variety7/12e4fd07dd3b0889ffa57e81262d515c: 605
BankBot/variety7/3317331e75aed00e9a26a7bd5cf83236: 613
BankBot/variety7/f120d71ebbfe390ecb84015248961069: 610
BankBot/variety7/ee8a99edea4053543dcb07172851cae1: 609
Bank

BankBot/variety7/ecf37d543490bf2875b808d6d97f904f: 603
BankBot/variety7/3c1e5c7916714ed442ffd7a27165cd40: 609
BankBot/variety7/d0525716bab0223807529a440ec04e23: 609
BankBot/variety7/7fb313a0f2ec97466d9ef493d4af41f1: 603
BankBot/variety7/1701c97b0ebf3cb86500ed8ca2decf43: 609
BankBot/variety7/9bb0c1e675e372a76061f958ada44c77: 1415
BankBot/variety7/98eab6743d32f8fa2992f05744a372ac: 609
BankBot/variety7/b91c57c8fb93bf1d9f54591287b07239: 1415
BankBot/variety7/43f5c943ec2ae45da0c43a85c387b892: 1415
BankBot/variety7/214b4adcfca01a264670d1a0387b01b3: 609
BankBot/variety7/731b6c3b289aa257341c9258c7994071: 605
BankBot/variety7/02cf263dd0079ead46a7dadb8835d3ef: 607
BankBot/variety7/2c7f1df2587bed4b82e2216591fcdb66: 603
BankBot/variety7/d0c79041d545347b29e899d54d3d6106: 603
BankBot/variety7/ef88f08d4296b3660e7b7b0e63dca626: 609
BankBot/variety7/f2762eb506c9cb7a2755f76ed3cb1901: 605
BankBot/variety7/6cf089fd36cd060dc59b9c7d663fa0db: 609
BankBot/variety7/3fa3c06554aaaa243beb67bcbce6fdc0: 1415
BankBo

BankBot/variety4/0e17c4ba54e2fd4828ffd60510840fd1: 59
BankBot/variety4/173d496e0219ff78c57926d92971a880: 39
BankBot/variety4/4b1905902fcb03e9dc8f1755e5ac62e2: 52
BankBot/variety4/9420a82313688c76e8bf327a904df5f2: 52
BankBot/variety4/00d471c121078c8c72ddbd205015b40b: 47
BankBot/variety4/c429ff0736c751c2f4ff2d95c5c1fb4c: 47
BankBot/variety3/ab817db57e14bb9f9601473edb081ebc: 220
BankBot/variety3/262ce98bd2bcdcd4e256c235c734104f: 46
BankBot/variety3/477dc44c5078a23ec8ddbb93248284a6: 910
BankBot/variety3/eb4e730a37574507efa677339893e056: 910
BankBot/variety3/1628689265986904e143ae1cffd5d797: 46
BankBot/variety3/371be8d1513051ba83cf3b036efbb51c: 220
BankBot/variety3/d7d911bc406c97fb2204f38a547cf5f4: 910
BankBot/variety3/7d6f03323bed8f1bcdc2593be78d4835: 910
BankBot/variety3/4e2b5eb7fc21fc5e3303aa25efcf8caf: 910
BankBot/variety3/7ce37eb373f999dce924adc996f4fa20: 220
BankBot/variety3/31e0b319f135da9a02626ea5894261fd: 910
BankBot/variety3/a30f8cf8a2e5b3981baccadaefed1b7a: 910
BankBot/variety3/5

BankBot/variety8/d9ab0461e557b13a4ad2d89523879177: 1462
BankBot/variety8/c1d0adda208d6dc69c0604b233f5840a: 1462
BankBot/variety8/76921d8255ad8791ed2ef5b1f85ca268: 1462
BankBot/variety8/708a8a647e0203504a247ad9c8ffa6da: 1462
BankBot/variety8/ac7a09db46dc16c2af0be6f4f7e333e1: 939
BankBot/variety8/fcbc8de7ec411cd22c41dd9ef54aece2: 938
BankBot/variety8/ba59fda101be38880403337df729fb16: 938
BankBot/variety8/0c7882720a3e6888e9cc695a01127966: 1462
BankBot/variety8/5e1114bb090ccee2c29208eb2df82ea0: 1462
BankBot/variety8/ab69c5c0b420fee018a4b78cc7432797: 938
BankBot/variety8/5d9d78a122b263eab20db8684e8a4afb: 938
BankBot/variety8/1836d0549bb8aa08d0231f67499139b3: 1462
BankBot/variety8/ec919aa8599dc34f6ea395e3c363d40e: 938
BankBot/variety8/5f5d5560aa0a6a1015913696803be299: 1462
BankBot/variety8/a7df093ff16fbac0cad5fea19c506b61: 1462
BankBot/variety8/1857ac8d4da075b6f1413f788689df5f: 1462
BankBot/variety8/bec73a10547fabfb1e5fdb23de8d03ae: 1462
BankBot/variety8/1a01d0afe84c9577a6943cc9a60e0e08: 146

BankBot/variety2/26a4085917ad4fd4865ad12d8520445b: 358
BankBot/variety2/f83eceef159ed225c6c88409ca1880f3: 358
BankBot/variety2/b6c53e0c9281c5ca019adf3c7ad72630: 358
BankBot/variety2/fb834d01719cb78704c86a8aaed9f800: 358
BankBot/variety2/6bdd6a7ac049fc5c8d0f97d8e1e1682f: 358
BankBot/variety2/17ce17a6eeaa8bc7d362cad66d5b2aaf: 358
BankBot/variety2/fafd579a3086649a76bd310d83c5c3b8: 358
BankBot/variety2/93350854655cd78b24abfe2c10326cfc: 48
BankBot/variety2/794dba6ac1f2b45065513ea4e6cfe167: 358
BankBot/variety2/9a599ed71af893a72628ec981fde7d02: 358
BankBot/variety2/7e84f07f690b2132fb33cb2cd45974fa: 358
BankBot/variety2/93aae8420cbe553cbcc9a6ee0e609302: 358
BankBot/variety2/262508ecf92848aa7b5c246a5154cd13: 358
BankBot/variety2/d6a854b15ff63d7c50f249810d615ce4: 358
BankBot/variety2/8fc5a9397e84ded85fa9503a1ced1cdc: 48
BankBot/variety2/0f5f2701322a9ae2742810fc1db9d576: 358
BankBot/variety5/558d9d20cdc432172a8eb79ad823e5bc: 1090
BankBot/variety5/2b1eb4ce1509abb3b509ccf36084698d: 1090
BankBot/va

Looking at average class amounts, it seems a lot of malware is simply repackaged and rebranded as a different app, when the internals seem the same. For example, Stealer has either 120 or 122 classes, with no other values being present. The same goes for many other apps, such as BankBot having 48, 348, 938, or 1462 a lot of the time. Due to some malware having as low as 19 classes, and others having over 1000, class amount is not really indicative of malware when compared to benign apps.

In [2]:
import os
import networkx as nx
from tqdm import tqdm_notebook
from collections import defaultdict
import pandas as pd

config = {}
# config['path_benign'] = '/datasets/dsc180a-wi20-public/Malware/testing/benign'
# config['path_benign'] = 'test'
config['path_benign'] = 'data'
config['path_malware'] = '/datasets/dsc180a-wi20-public/Malware/testing/malware'

api_list = []
app_list = []
package_list = []
seen_api = set()
# api_counts = defaultdict(int)

# pandas method
# data = pd.DataFrame()

app_to_api = nx.Graph()
api_cooccur = nx.Graph()
api_same_invoke = nx.Graph()
api_same_package = nx.Graph()

for directory in tqdm_notebook(next(os.walk(config['path_benign']))[1]):
    app_list.append(directory)
    app_to_api.add_node(directory)
    
    # pandas method
#     app_data = []
    
    for subdir, dirs, files in os.walk(config['path_benign'] + '/' + directory):
        for i, file in enumerate(files):
            if i == 100:
                break
            filepath = subdir + os.sep + file

            with open(filepath, 'r') as fp:
                in_method = ''
                api_calls = set()
                
                for line in fp:
                    stripped = line.strip()
                    if stripped.startswith('.method'):
                        in_method = stripped[8:]
                    if stripped == '.end method':
                        in_method = ''
                        api_calls.clear()
                    if stripped[:6] == 'invoke':
                        invoke_method = stripped.split(' {')[0][7:].split('/')[0]
                        
                        splitted = line.split('}, ')
                        fns = splitted[1].split('->')
                        api_package = fns[0]
                        method = fns[1].split('(')[0]
                        
                        current_method_call = api_package + ',' + method
    
                        api_calls.add(current_method_call)
                        
                        # pandas method
#                         app_data.append([directory, invoke_method, '%s,%s,%s' % (directory, file, in_method), api_package, current_method_call])
    
                        if current_method_call not in seen_api:
                            api_list.append(current_method_call)
                            seen_api.add(current_method_call)
                    
                        # app to api generation
                        if current_method_call not in app_to_api:
                            app_to_api.add_node(current_method_call)
                        app_to_api.add_edge(directory, current_method_call)
                        
                        # api to api co-occurance generation
                        if in_method != '':
                            if current_method_call not in api_cooccur:
                                api_cooccur.add_node(current_method_call)
                            for api_call in api_calls:
                                if not api_cooccur.has_edge(current_method_call, api_call):
                                    api_cooccur.add_edge(current_method_call, api_call)
                                
                        # api to api invoke generation
                        # can go from invoke method <-> method call to method call <-> method call by doing A*(A^T)
                        if invoke_method not in api_same_invoke:
                            api_same_invoke.add_node(invoke_method)
                        if current_method_call not in api_same_invoke:
                            api_same_invoke.add_node(current_method_call)
                        if not api_same_invoke.has_edge(invoke_method, current_method_call):
                            api_same_invoke.add_edge(invoke_method, current_method_call)
                                
                        # api to api package generation
                        # can go from package <-> method call to method call <-> method call by doing A*(A^T)
                        if api_package not in api_same_package:
                            package_list.append(api_package)
                            api_same_package.add_node(api_package)
                        if current_method_call not in api_same_package:
                            api_same_package.add_node(current_method_call)
                        if not api_same_package.has_edge(api_package, current_method_call):
                            api_same_package.add_edge(api_package, current_method_call)
    # pandas method
#     data = data.append(app_data)
# data.columns = ['app', 'invoke', 'method', 'package', 'api']
# nx.draw(app_to_api)

In [33]:
# api-to-api same invoke
from scipy.sparse import csr_matrix
import scipy.sparse
import numpy as np
invoke_types = ['direct', 'static', 'virtual', 'super', 'interface']
matrix_P = nx.adjacency_matrix(api_same_invoke, nodelist=api_list + invoke_types)[-len(invoke_types):, :-len(invoke_types)]
# scipy.sparse.save_npz('matrix_C.npz', matrix_C)
# matrix_C_disk = np.memmap('matrix_C.npz', dtype='float32', mode='r')
# matrix_C_disk.shape
# matrix_C_disk_final = np.memmap('matrix_C_final.npz', dtype='float32', mode='w+', shape=(matrix_C_disk.shape[1], matrix_C_disk.shape[1]))
# matrix_C_disk_final[:] = matrix_C_disk.T.dot(matrix_C_disk)
matrix_P = matrix_P.transpose() @ matrix_P
# matrix_C = csr_matrix(np.matmul(matrix_C.T, matrix_C))
matrix_P

<17656x17656 sparse matrix of type '<class 'numpy.int64'>'
	with 103752202 stored elements in Compressed Sparse Column format>

In [34]:
# api-to-api same package
matrix_I = nx.adjacency_matrix(api_same_package, nodelist=api_list + package_list)[-len(package_list):, :-len(package_list)]
matrix_I = matrix_D.transpose() @ matrix_I
matrix_I

<17656x17656 sparse matrix of type '<class 'numpy.int64'>'
	with 239330 stored elements in Compressed Sparse Column format>

# pandas implementation for deriving matrices

In [ ]:
# pandas app-api
from scipy.sparse import csr_matrix
matrix_A = csr_matrix(pd.pivot_table(data, index='app', columns='api', aggfunc='count').fillna(0).clip(0, 1).values)
matrix_A

In [12]:
# pandas api-api same package
from scipy.sparse import csr_matrix
import numpy as np
part_package = csr_matrix(pd.pivot_table(data, index='api', columns='package', aggfunc='count').fillna(0).clip(0, 1).values)
matrix_package = part_package @ part_package.transpose()
matrix_package

<17656x17656 sparse matrix of type '<class 'numpy.float64'>'
	with 239330 stored elements in Compressed Sparse Row format>

# Analysis on benign apps

In [8]:
# O(n + k) top kth frequent appearing elements, sorry heap but O(nlogk) is just too slow for leetcode

freq_map = defaultdict(list)
max_degree = 0
for node in app_to_api.degree:
    if max_degree < node[1]:
        max_degree = node[1]
    freq_map[node[1]].append(node[0])

highest_k = 100
should_break = False
for i in range(len(app_list), -1, -1):
    if i in freq_map:
        for api in freq_map[i]:
            print('%s: %i' % (api, i))
            highest_k -= 1
            if highest_k == 0:
                should_break = True
                break
    if should_break:
        break

Ljava/lang/Object;,<init>: 98
Landroid/app/Activity;,<init>: 98
Ljava/lang/StringBuilder;,<init>: 97
Ljava/lang/StringBuilder;,append: 97
Ljava/lang/StringBuilder;,toString: 97
Ljava/lang/Integer;,valueOf: 97
Ljava/util/ArrayList;,<init>: 97
Ljava/util/List;,size: 97
Ljava/util/Iterator;,hasNext: 97
Ljava/util/Iterator;,next: 97
Landroid/content/Context;,getPackageName: 97
Ljava/util/ArrayList;,add: 97
Ljava/lang/System;,arraycopy: 97
Ljava/lang/RuntimeException;,<init>: 97
Landroid/content/Intent;,<init>: 97
Ljava/lang/Object;,getClass: 97
Landroid/net/Uri;,parse: 97
Ljava/lang/Boolean;,valueOf: 97
Ljava/util/Locale;,getDefault: 97
Ljava/util/List;,iterator: 97
Landroid/app/Activity;,onCreate: 97
Landroid/content/Context;,getSystemService: 96
Landroid/view/WindowManager;,getDefaultDisplay: 96
Ljava/lang/Math;,min: 96
Ljava/lang/IllegalArgumentException;,<init>: 96
Landroid/graphics/Rect;,<init>: 96
Landroid/content/res/Resources;,getDisplayMetrics: 96
Landroid/util/Log;,w: 96
Ljava/la

The app with the highest amount of API calls in my sample set is a cryptocurrency wallet, with a total of 63992 unique API calls. The API with the highest usages are core Java utils, used among all 50 apps. These include things such as initialization of Object (from super() calls due to inheritance), StringBuilders, Java Collection objects, and IOStreams. However, these are boring, so above shown are only APIs that are called at most (# of apps - 1) times. In the same vein, the code below shows the least-used 100 APIs that are used by at least 2 apps. Many of these APIs have to do with ads, or analytics.

In [10]:
highest_k = 100
should_break = False
for i in range(2, len(app_list), 1):
    if i in freq_map:
        for api in freq_map[i]:
            print('%s: %i' % (api, i))
            highest_k -= 1
            if highest_k == 0:
                should_break = True
                break
    if should_break:
        break

[Lcom/google/ads/b;,clone: 2
[Lcom/google/ads/c;,clone: 2
Lcom/google/ads/mediation/d;,b: 2
Lcom/google/ads/mediation/d;,c: 2
Lcom/google/ads/mediation/d;,d: 2
Lcom/google/android/gms/ads/mediation/MediationBannerListener;,a: 2
Lcom/google/android/gms/ads/mediation/MediationBannerListener;,b: 2
Lcom/google/android/gms/ads/mediation/MediationBannerListener;,d: 2
Lcom/google/android/gms/ads/mediation/MediationBannerListener;,c: 2
Lcom/google/android/gms/ads/mediation/MediationBannerListener;,e: 2
Lcom/google/android/gms/ads/mediation/MediationInterstitialListener;,b: 2
Lcom/google/ads/mediation/c;,<init>: 2
Lcom/google/ads/mediation/d;,<init>: 2
Lcom/google/android/gms/ads/mediation/MediationAdRequest;,a: 2
Lcom/google/android/gms/ads/mediation/MediationAdRequest;,b: 2
Lcom/google/android/gms/ads/mediation/MediationAdRequest;,c: 2
Lcom/google/android/gms/ads/mediation/MediationAdRequest;,d: 2
Lcom/google/android/gms/ads/internal/client/zzn;,a: 2
Lcom/google/android/gms/ads/internal/util/

# Malware
To do malware analysis here, one variant of each malware type will be parsed.

In [3]:
import os
import networkx as nx
from tqdm import tqdm_notebook
from collections import defaultdict
import pandas as pd

config = {}
# config['path_benign'] = '/datasets/dsc180a-wi20-public/Malware/testing/benign'
# config['path_benign'] = 'test'
config['path_benign'] = 'data'
config['path_malware'] = '/datasets/dsc180a-wi20-public/Malware/amd_data_smali'
# api_counts = defaultdict(int)

# pandas method
# data = pd.DataFrame()

# app_to_api = nx.Graph()
# api_cooccur = nx.Graph()
# api_same_invoke = nx.Graph()
# api_same_package = nx.Graph()

for directory in tqdm_notebook(next(os.walk(config['path_malware']))[1]):
    app_list.append(directory)
    app_to_api.add_node(directory)
    
    for subdir, dirs, files in os.walk(config['path_malware'] + '/' + directory + '/' + os.listdir(config['path_malware'] + '/' + directory)[0]):
        for i, file in enumerate(files):
            if i == 100:
                break
            filepath = subdir + os.sep + file

            with open(filepath, 'r') as fp:
                in_method = ''
                api_calls = set()

                for line in fp:
                    stripped = line.strip()
                    if stripped.startswith('.method'):
                        in_method = stripped[8:]
                    if stripped == '.end method':
                        in_method = ''
                        api_calls.clear()
                    if stripped[:6] == 'invoke':
                        invoke_method = stripped.split(' {')[0][7:].split('/')[0]

                        splitted = line.split('}, ')
                        fns = splitted[1].split('->')
                        api_package = fns[0]
                        method = fns[1].split('(')[0]

                        current_method_call = api_package + ',' + method

                        api_calls.add(current_method_call)

                        # pandas method
#                         app_data.append([directory, invoke_method, '%s,%s,%s' % (directory, file, in_method), api_package, current_method_call])

                        if current_method_call not in seen_api:
                            api_list.append(current_method_call)
                            seen_api.add(current_method_call)

                        # app to api generation
                        if current_method_call not in app_to_api:
                            app_to_api.add_node(current_method_call)
                        app_to_api.add_edge(directory, current_method_call)

                        # api to api co-occurance generation
                        if in_method != '':
                            if current_method_call not in api_cooccur:
                                api_cooccur.add_node(current_method_call)
                            for api_call in api_calls:
                                if not api_cooccur.has_edge(current_method_call, api_call):
                                    api_cooccur.add_edge(current_method_call, api_call)

                        # api to api invoke generation
                        # can go from invoke method <-> method call to method call <-> method call by doing A*(A^T)
                        if invoke_method not in api_same_invoke:
                            api_same_invoke.add_node(invoke_method)
                        if current_method_call not in api_same_invoke:
                            api_same_invoke.add_node(current_method_call)
                        if not api_same_invoke.has_edge(invoke_method, current_method_call):
                            api_same_invoke.add_edge(invoke_method, current_method_call)

                        # api to api package generation
                        # can go from package <-> method call to method call <-> method call by doing A*(A^T)
                        if api_package not in api_same_package:
                            package_list.append(api_package)
                            api_same_package.add_node(api_package)
                        if current_method_call not in api_same_package:
                            api_same_package.add_node(current_method_call)
                        if not api_same_package.has_edge(api_package, current_method_call):
                            api_same_package.add_edge(api_package, current_method_call)
    # pandas method
#     data = data.append(app_data)
# data.columns = ['app', 'invoke', 'method', 'package', 'api']
# nx.draw(app_to_api)

In [3]:
# O(n + k) top kth frequent appearing elements, sorry heap but O(nlogk) is just too slow for leetcode

freq_map = defaultdict(list)
max_degree = 0
for node in app_to_api.degree:
    if max_degree < node[1]:
        max_degree = node[1]
    freq_map[node[1]].append(node[0])

highest_k = 100
should_break = False
for i in range(len(app_list), -1, -1):
    if i in freq_map:
        for api in freq_map[i]:
            print('%s: %i' % (api, i))
            highest_k -= 1
            if highest_k == 0:
                should_break = True
                break
    if should_break:
        break

Ljava/lang/Object;,<init>: 33
Ljava/lang/StringBuilder;,<init>: 33
Ljava/lang/StringBuilder;,append: 33
Ljava/lang/StringBuilder;,toString: 33
Ljava/lang/String;,equals: 33
Landroid/os/Handler;,<init>: 33
Landroid/app/Activity;,<init>: 33
Landroid/app/Activity;,onCreate: 33
Ljava/util/ArrayList;,<init>: 32
Ljava/lang/String;,length: 32
Ljava/lang/String;,substring: 32
Ljava/lang/Integer;,valueOf: 32
Ljava/util/Iterator;,hasNext: 32
Ljava/util/Iterator;,next: 32
Landroid/content/Intent;,<init>: 32
Ljava/lang/System;,currentTimeMillis: 32
Ljava/util/List;,get: 31
Landroid/content/Context;,getSystemService: 31
Ljava/lang/Integer;,intValue: 31
Landroid/content/BroadcastReceiver;,<init>: 31
Landroid/content/Intent;,putExtra: 31
Ljava/lang/Integer;,parseInt: 31
Landroid/net/Uri;,parse: 31
Landroid/util/Log;,d: 31
Landroid/util/Log;,e: 30
Ljava/util/List;,add: 30
Ljava/lang/reflect/Method;,invoke: 30
Ljava/lang/Thread;,<init>: 30
Ljava/lang/String;,startsWith: 30
Ljava/lang/Object;,getClass: 

In [4]:
highest_k = 100
should_break = False
for i in range(2, len(app_list), 1):
    if i in freq_map:
        for api in freq_map[i]:
            print('%s: %i' % (api, i))
            highest_k -= 1
            if highest_k == 0:
                should_break = True
                break
    if should_break:
        break

Landroid/graphics/drawable/Drawable$ConstantState;,getChangingConfigurations: 2
Landroid/support/v4/content/res/ResourcesCompat;,getDrawable: 2
Landroid/animation/AnimatorSet;,getChildAnimations: 2
Landroid/animation/ObjectAnimator;,getPropertyName: 2
Landroid/animation/ArgbEvaluator;,<init>: 2
Landroid/animation/ObjectAnimator;,setEvaluator: 2
Landroid/support/v4/graphics/drawable/DrawableCompat;,getAlpha: 2
Landroid/support/v4/graphics/drawable/DrawableCompat;,inflate: 2
Landroid/animation/AnimatorInflater;,loadAnimator: 2
Landroid/graphics/drawable/AnimatedVectorDrawable;,isRunning: 2
Landroid/graphics/drawable/AnimatedVectorDrawable;,start: 2
Landroid/graphics/drawable/AnimatedVectorDrawable;,stop: 2
Landroid/graphics/drawable/Drawable;,onLevelChange: 2
Landroid/graphics/PathMeasure;,setPath: 2
Landroid/graphics/PathMeasure;,getSegment: 2
Landroid/support/v4/graphics/ColorUtils;,getTempDouble3Array: 2
Landroid/support/v4/graphics/ColorUtils;,LABToXYZ: 2
Landroid/support/v4/graphics

Looking at malware used APIs, some APIs immediately jump to the forefront. There are uses of os-level APIs from android.os, as well as usage of typed arrays, probably for some sort of buffer overflow attack. Commonly used APIs include setting shared preferences, hex strings, file operations, and intent. Less commonly-used APIs for malware do not include ad-related APIs like benign apps. Most of these APIs seem to be for backwards graphical support.

In [4]:
# testapp-api @ api-api @ api-testapp -> score for models
# app-api
matrix_A = nx.adjacency_matrix(app_to_api, api_list + app_list)[-len(app_list):, :-len(app_list)]

# api-api co-occur
matrix_B = nx.adjacency_matrix(api_cooccur, api_list)

matrix_A, matrix_B

(<132x1163654 sparse matrix of type '<class 'numpy.int64'>'
 	with 2756767 stored elements in Compressed Sparse Row format>,
 <1163654x1163654 sparse matrix of type '<class 'numpy.int64'>'
 	with 37935736 stored elements in Compressed Sparse Row format>)

In [5]:
from sklearn.linear_model import LogisticRegression
import scipy.sparse
app_to_app = matrix_A @ matrix_B @ matrix_A.transpose()
# scipy.sparse.save_npz('matrix_ABAT.npz', app_to_app)
# app_to_app = scipy.sparse.load_npz('matrix_ABAT.npz')
y_list = [1] * len(app_list[:-33]) + [0] * len(app_list[-33:])

In [6]:
logistic = LogisticRegression()
logistic.fit(app_to_app[5:-5], y_list[5:-5])
logistic.score(app_to_app[:5], y_list[:5]), logistic.score(app_to_app[-5:], y_list[-5:])

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.8, 1.0)

With a simple logistic regression, 94 benign samples, and 28 malware samples, there is an accuracy of 80% on scoring benign apps as benign, and all 5 malware samples were identified as malware. The commuting matrix used was ABA<sup>T</sup>. This commuting matrix was used because B is the co-occur matrix, so apps that use the same APIs in one method are probably closely related in how they function.

In [7]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(app_to_app[5:-5], y_list[5:-5])
rfc.score(app_to_app[:5], y_list[:5]), logistic.score(app_to_app[-5:], y_list[-5:])

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


(1.0, 1.0)

In [10]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_jobs=8, booster='dart', tree_method='exact', n_estimators=200)
xgb.fit(app_to_app[5:-5], y_list[5:-5])
xgb.score(app_to_app[:5], y_list[:5]), logistic.score(app_to_app[-5:], y_list[-5:])

(1.0, 1.0)

Using a random forest and xgboost library both led to perfect accuracy scores on benign and malware. However, this is a limited test set and such results aren't indicative of all different varieties of malware and benign apps. Most apps collected here were games, so productivity apps and other categories are left out and may be categorized as malware accidentally.